<a href="https://colab.research.google.com/github/MhammadRezaValiYari/GoldTransectionAnalysisLinearRegression/blob/main/GoldTransectionAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import essesional library

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import sklearn.model_selection

read data from drive

In [8]:
data = yf.download('GC=F', start='2023-12-19', end='2024-12-19')

[*********************100%***********************]  1 of 1 completed


In [9]:
data.head(2)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,GC=F,GC=F,GC=F,GC=F,GC=F,GC=F
Date,,,,,,
2023-12-19,2038.400024,2038.400024,2042.699951,2025.199951,2025.199951,498
2023-12-20,2034.500000,2034.500000,2037.099976,2030.300049,2037.099976,258


In [30]:
short_window = 50
long_window = 200

data['SMA50'] = data['Close'].rolling(window=50).mean()
data['sma200'] = data['Close'].rolling(window=200).mean()

In [31]:
series = data[['Close', 'SMA50', 'sma200']]

In [32]:
series.head(2)

Price,Close,SMA50,sma200
Ticker,GC=F,,
Date,,,
2023-12-19,2038.400024,NaN,NaN
2023-12-20,2034.500000,NaN,NaN


In [33]:
series['SMA50']

,SMA50
Date,
2023-12-19,NaN
2023-12-20,NaN
2023-12-21,NaN
2023-12-22,NaN
2023-12-26,NaN
...,...
2024-12-12,NaN
2024-12-13,NaN
2024-12-16,NaN
